In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

name = "dwightf/berkshireGPT"
     
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(name, quantization_config=bnb_config)


tokenizer = AutoTokenizer.from_pretrained(name, use_fast=True, device_map="auto")



In [ ]:
while True:
    print("Type 'quit' to exit")
    question = input("Enter question: ")
    if question == "quit":
        break
        
    eval_prompt = f"""### Instruction:
    You are a value investor giving your advice on stocks. And choosing whether to buy, sell, or hold them.
    ### Text:
    Q -{question}
    
    """
    
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    model.eval()
    with torch.no_grad():
    
        tokenizer.decode(model.generate(**model_input, streamer=streamer,
                            use_cache=True, max_new_tokens=float('inf'))[0], skip_special_tokens=True)
        